In [22]:
import folium
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from branca.element import Template, MacroElement



bus_df = pd.read_csv("노선_요약.csv")
subway_df = pd.read_csv("지하철_요약.csv")
cafe_df = pd.read_csv("일반음식점(카페)현황.csv", encoding='CP949')
park_df = pd.read_csv("도시공원정보현황(제공표준).csv", encoding='CP949')
trash_bin_df = pd.read_csv("경기도 성남시_쓰레기통_설치현황_20250325.csv")

cafe_df = cafe_df[(cafe_df['시군명'] == '성남시') & (cafe_df['영업상태명'] == '영업')]
cafe_df = cafe_df.rename(columns = {'WGS84위도' : '위도','WGS84경도': '경도'})
# 시설규모 바탕으로 가중치 생성
scaler = MinMaxScaler()
cafe_df['가중치'] = scaler.fit_transform(cafe_df[['총시설규모(㎡)']])
cafe_df

park_df = park_df[park_df['소재지지번주소'].str.startswith('경기도 성남시')]
park_df['가중치'] = scaler.fit_transform(park_df[['공원면적(㎡)']])
park_df = park_df.dropna(subset=['위도', '경도'])
park_df.isnull().sum()


bus_df = bus_df.dropna()
bus_df['혼잡도'] = bus_df['환승시간(분)']*bus_df['노선개수']
bus_df['가중치'] = scaler.fit_transform(bus_df[['혼잡도']])
bus_df

subway_df['일평균승하차인원'] = subway_df['승차총승객수'] + subway_df['하차총승객수']
subway_df['가중치'] = scaler.fit_transform(subway_df[['일평균승하차인원']])
subway_df

# 이름 통일
bus_df = bus_df.rename(columns = {'정류장명':'이름'}) 
subway_df = subway_df.rename(columns = {'역사명':'이름'}) 
cafe_df = cafe_df.rename(columns = {'사업장명':'이름'}) 
park_df = park_df.rename(columns = {'공원명':'이름'}) 

# 필요한 열만 추출
bus_coords = bus_df[['위도', '경도','가중치','이름']].copy()
subway_coords = subway_df[['위도', '경도','가중치','이름']].copy()
cafe_coords = cafe_df[['위도', '경도','가중치','이름']].copy()
park_coords = park_df[['위도', '경도','가중치','이름']].copy()

# 두 데이터프레임을 위로 붙이기
all_df = pd.concat([bus_coords, subway_coords,cafe_coords,park_coords], ignore_index=True)
all_df

candidate_df = pd.concat([bus_df, park_df], ignore_index=True)


# 지도 중심은 첫 번째 버스 정류장
m = folium.Map(location=[bus_df['위도'].iloc[0], bus_df['경도'].iloc[0]], zoom_start=14)

# 버스 정류장 시각화 
for _, row in bus_df.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=5,
        popup=row['이름'],
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# 지하철 역사 시각화
for _, row in subway_df.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=6,
        popup=row['이름'],
        color='yellow',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# 카페 시각화 
for _, row in cafe_df.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=6,
        popup=row['이름'],
        color='grey',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# 공원 시각화 
for _, row in park_df.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=6,
        popup=row['이름'],
        color='green',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# 쓰레기통 시각화 
trash_bin_df = trash_bin_df.dropna(subset=['위도', '경도'])  # 위경도 누락 제거

for _, row in trash_bin_df.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=4,
        popup=row.get('설치장소', '쓰레기통'),  # 없으면 '쓰레기통'으로 대체
        color='red',
        fill=True,
        fill_opacity=0.6
    ).add_to(m)


# # 각 쓰레기통 반경 500m 커버 범위 시각화 (투명 원)
# for _, row in trash_bin_df.iterrows():
#     folium.Circle(
#         location=[row['위도'], row['경도']],
#         radius=1000,  # 반경 500m
#         color='red',
#         fill=True,
#         fill_opacity=0.1  # 살짝 투명하게
#     ).add_to(m)


title_html = '''
    <h3 align="center" style="font-size:20px"><b>1000m Coverage of Trash Bins in Seongnam</b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))
legend_html = """
{% macro html(this, kwargs) %}

<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 150px; height: 150px; 
    background-color: white; z-index:9999; 
    font-size:14px; border:2px solid grey; 
    padding: 10px;
    ">
    <b>범례</b><br>
    <i style="background: blue; width:10px; height:10px; float:left; margin-right:5px;"></i> 버스 정류장<br>
    <i style="background: yellow; width:10px; height:10px; float:left; margin-right:5px;"></i> 지하철역<br>
    <i style="background: grey; width:10px; height:10px; float:left; margin-right:5px;"></i> 카페<br>
    <i style="background: green; width:10px; height:10px; float:left; margin-right:5px;"></i> 공원<br>
    <i style="background: red; width:10px; height:10px; float:left; margin-right:5px;"></i> 기존 쓰레기통<br>

</div>

{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# 최종 지도 출력
# m.save("성남시_전체.html")
m
